In [0]:
%pip install -U -qqqq unitycatalog-ai[databricks] mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents python-dotenv uv
%restart_python

In [0]:
import sys
sys.path.append(".")


In [0]:
dbutils.widgets.removeAll()

In [0]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())  

# Create widgets in order to bind environment variables to the SQL session variables so they can be used in %sql cells
dbutils.widgets.text("TARGET_CATALOG", os.getenv("TARGET_CATALOG"))
dbutils.widgets.text("TARGET_SCHEMA", os.getenv("TARGET_SCHEMA"))
dbutils.widgets.text("VS_INDEX_BASE_TABLE", os.getenv("VS_INDEX_BASE_TABLE"))
dbutils.widgets.text("VS_INDEX", os.getenv("VS_INDEX"))

TARGET_CATALOG = dbutils.widgets.get("TARGET_CATALOG")
TARGET_SCHEMA = dbutils.widgets.get("TARGET_SCHEMA")
VS_INDEX_BASE_TABLE = dbutils.widgets.get("VS_INDEX_BASE_TABLE")
VS_INDEX = dbutils.widgets.get("VS_INDEX")

print(f"TARGET_CATALOG: {TARGET_CATALOG}")
print(f"TARGET_SCHEMA: {TARGET_SCHEMA}")
print(f"VS_INDEX_BASE_TABLE: {VS_INDEX_BASE_TABLE}")
print(f"VS_INDEX: {VS_INDEX}")

In [0]:
from databricks.vector_search.client import VectorSearchClient

client = VectorSearchClient()

index_name = f"{TARGET_CATALOG}.{TARGET_SCHEMA}.{VS_INDEX}"
print(index_name)

if index_name not in [
    i["name"] for i in client.list_indexes(TARGET_CATALOG).get("vector_indexes", [])
]:
  print(f"Creating index {index_name}")
  client.create_delta_sync_index_and_wait(
        endpoint_name=TARGET_CATALOG,
        source_table_name=f"{TARGET_CATALOG}.{TARGET_SCHEMA}.{VS_INDEX_BASE_TABLE}",
        index_name=index_name,
        pipeline_type="TRIGGERED",
        primary_key="product_id",
        embedding_source_column="product_doc",
        embedding_model_endpoint_name="databricks-gte-large-en",
    )
print(f"Created index {index_name}")